**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
#skvideo.setFFmpegPath("C:/Users/david/Anaconda3/envs/deeplearning/Lib/site-packages/skvideo/io")
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd,Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten,Dropout

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

***
__Answer__

The function act define which action the agent is going to do next.
<br> Epsilon is the symbol of an Epsilon-greedy policy. At each state the agent chooses what it believes to be the best action, but sometimes he also has to acts randomly.

This way the agent takes actions which he may not estimate to be ideal, but may provide new information to the agent.
<br>The Espilon determines the probability of taking a random action.

<br> We draw a number between 0 and 1. If this number > epsilon, then we will do “exploitation” (this means we use what we already know to select the best action at each step). Else, we’ll do exploration.

***
### The Game

#The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)



epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()


# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))     #Tableau avec rewards : poison -1 /cheese +0.5
        self.position = np.zeros((grid_size,grid_size))  #Position of the rat

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3)) #(502,10*16,10*16,3)


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0: #Down
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: #Up
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: #Right
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: #Left 
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0] # The rat starts anywhere on the board
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2) #binomial with p=temperature
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

***
__Answer__

The array Board contains every rewards the rat can get. It has the same size than position (size = `grid_size` $\times$ `grid_size`). Every cell with cheese has a reward of +0.5 and those with poison has a reward of -1.

The array position gives the position of the rat, (x,y) corresponds to its coordinates in line and in column.


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        action = np.random.randint(0,4)       
        pass

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
             # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
    # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward 

        # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over) 

        
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature) #Sets the environment 

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random') #Play the game until Game Over for each epoch
HTML(display_videos('random0.mp4'))

Win/lose count 1.5/2.0. Average score (-0.5)
Win/lose count 2.0/3.0. Average score (-0.75)
Win/lose count 1.0/3.0. Average score (-1.1666666666666667)
Win/lose count 1.5/1.0. Average score (-0.75)
Win/lose count 1.0/2.0. Average score (-0.8)
Win/lose count 1.5/5.0. Average score (-1.25)
Win/lose count 1.5/2.0. Average score (-1.1428571428571428)
Win/lose count 2.0/1.0. Average score (-0.875)
Win/lose count 2.0/0. Average score (-0.5555555555555556)
Win/lose count 2.5/0. Average score (-0.25)
Win/lose count 2.0/4.0. Average score (-0.4090909090909091)
Win/lose count 1.0/3.0. Average score (-0.5416666666666666)
Win/lose count 1.0/4.0. Average score (-0.7307692307692307)
Win/lose count 2.0/1.0. Average score (-0.6071428571428571)
Win/lose count 0.5/1.0. Average score (-0.6)
Win/lose count 0/1.0. Average score (-0.625)
Win/lose count 1.0/3.0. Average score (-0.7058823529411765)
Win/lose count 1.5/2.0. Average score (-0.6944444444444444)
Win/lose count 1.5/2.0. Average score (-0.68421052631

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
__Proof__ 

From the definition, and using the properties of the conditional expection,
$$
\begin{aligned}
Q^{\pi}(s,a) &= E_{\pi}[ \sum_{t=0}^{\infty}\gamma^t r(s_t,a_t) \ | \ s_0 = s, a_0 = a]\\
&= E[r(s,a)]+ E_{\pi}[ \sum_{t=1}^{\infty}\gamma^{t} r(s_{t},a_{t}) \ | \ s_0 = s, a_0 = a ]    \\
&= E[r(s,a)]+ \gamma \sum_{s'\in S}^{} \sum_{a' \in A}^{} p(s_1 = s',a_1 = a'|s_0 = s, a_0 = a)E_{\pi}[\sum_{t=1}^{\infty}\gamma^{t-1} r(s_{t},a_{t}) \ |\  s_1 = s',a_1 = a' ]\\
& = E[r(s,a)]+ \gamma \sum_{s'\in S}^{} \sum_{a' \in A}^{} p(s_1 = s',a_1 = a'|s_0 = s, a_0 = a)Q^{\pi}(s',a')\\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{aligned}
$$



Then, for the optimal policy $\pi^*$, the Q-function is $Q^{*}(s,a) = max_{\pi} Q^{\pi}(s,a)$.
Using what we just prove we get:
$$
\begin{aligned}
Q^*(s,a) & = \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
 & = r(s,a) + \gamma \max_{\pi} [ \sum_{s' \in S}^{}  \sum_{a' \in A}^{} p(s',a'|s,a)Q^{\pi}(s',a') ] \\
 & = r(s,a) + \gamma  \sum_{s' \in S}^{}p(s'|s,a)
 \max_{\pi} [\sum_{a' \in A}^{} \pi(a'|s')Q^{\pi}(s',a') ]\\
 & \text{We have} ~~
 max_{a, \pi’} [f(a)+g(\pi’)] = max_a f(a)+ max_{\pi’} g(\pi’) \\ 
 & Hence: \\
 & = r(s,a) + \gamma  \sum_{s' \in S}^{}p(s'|s,a) \max_{a'} Q^*(s',a') \\
& = r(s,a) + \gamma  \sum_{s' \in S}^{}p(s'|s,a) \max_{a'} Q^*(s',a')\\
& = E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{aligned}
$$


$$
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}$$ is exactly the MSE which is a pretty accurate loss function.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
import random
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0) 
            
    def random_access(self):
        return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats 
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size


    def learned_act(self, s):
        predicted_Q = self.model.predict(s.reshape(1,5,5,self.n_state))
        return np.argmax(predicted_Q)
    

    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        # First, memorize the states
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s, n_s, a, r, game_over = self.memory.random_access()
            pred = self.model.predict(np.array([s, n_s]))
            target_q[i] = pred[0]
            input_states[i] = s
            if game_over_:
                target_q[i][a] = r
            else:
                target_q[i][a] = r + self.discount*max(pred[1])

        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args,**kwargs):
        lr = kwargs.pop('lr', 0.1)
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(32,activation='tanh'))
        model.add(Dense(4))
        model.compile(Adam(lr=lr), "mse")
        
        self.model = model

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, epochs_train, prefix='fc_train') # Training to get best Q
HTML(display_videos('fc_train10.mp4'))

Epoch 000/030 | Loss 0.8401 | Win/lose count 3.5/1.0 (2.5)
Epoch 001/030 | Loss 1.7679 | Win/lose count 5.0/4.0 (1.0)
Epoch 002/030 | Loss 2.0403 | Win/lose count 4.5/6.0 (-1.5)
Epoch 003/030 | Loss 1.9786 | Win/lose count 1.5/4.0 (-2.5)
Epoch 004/030 | Loss 1.9494 | Win/lose count 4.5/0 (4.5)
Epoch 005/030 | Loss 1.9639 | Win/lose count 4.0/4.0 (0.0)
Epoch 006/030 | Loss 2.0325 | Win/lose count 6.0/7.0 (-1.0)
Epoch 007/030 | Loss 2.0991 | Win/lose count 3.0/1.0 (2.0)
Epoch 008/030 | Loss 2.0421 | Win/lose count 6.5/2.0 (4.5)
Epoch 009/030 | Loss 2.0238 | Win/lose count 6.0/3.0 (3.0)
Epoch 010/030 | Loss 1.9721 | Win/lose count 3.0/2.0 (1.0)
Epoch 011/030 | Loss 1.9898 | Win/lose count 3.5/2.0 (1.5)
Epoch 012/030 | Loss 2.0434 | Win/lose count 6.0/0 (6.0)
Epoch 013/030 | Loss 2.0516 | Win/lose count 5.5/0 (5.5)
Epoch 014/030 | Loss 2.0302 | Win/lose count 5.0/1.0 (4.0)
Epoch 015/030 | Loss 2.0103 | Win/lose count 3.5/3.0 (0.5)
Epoch 016/030 | Loss 2.0371 | Win/lose count 4.5/3.0 (1.5)


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(128, (3,3), activation='relu',input_shape=(5,5,self.n_state)))
        model.add(Conv2D(64, (3,3), activation='tanh'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(Adam(lr=lr), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/030 | Loss 1.7262 | Win/lose count 1.5/3.0 (-1.5)
Epoch 001/030 | Loss 2.1301 | Win/lose count 10.0/9.0 (1.0)
Epoch 002/030 | Loss 2.0455 | Win/lose count 5.5/7.0 (-1.5)
Epoch 003/030 | Loss 2.0010 | Win/lose count 11.0/5.0 (6.0)
Epoch 004/030 | Loss 1.9981 | Win/lose count 8.5/2.0 (6.5)
Epoch 005/030 | Loss 2.0519 | Win/lose count 10.0/2.0 (8.0)
Epoch 006/030 | Loss 1.9781 | Win/lose count 2.5/3.0 (-0.5)
Epoch 007/030 | Loss 2.1148 | Win/lose count 12.5/5.0 (7.5)
Epoch 008/030 | Loss 2.1050 | Win/lose count 10.0/2.0 (8.0)
Epoch 009/030 | Loss 2.0096 | Win/lose count 6.5/3.0 (3.5)
Epoch 010/030 | Loss 2.0109 | Win/lose count 9.0/3.0 (6.0)
Epoch 011/030 | Loss 2.0359 | Win/lose count 13.0/1.0 (12.0)
Epoch 012/030 | Loss 2.1432 | Win/lose count 5.5/6.0 (-0.5)
Epoch 013/030 | Loss 1.9788 | Win/lose count 14.0/4.0 (10.0)
Epoch 014/030 | Loss 2.1121 | Win/lose count 10.0/1.0 (9.0)
Epoch 015/030 | Loss 1.8511 | Win/lose count 20.0/5.0 (15.0)
Epoch 016/030 | Loss 2.0476 | Win/lose c

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.4) 
# temperature = 0.3 Averages 7.8 CNN & 3.9 FC // T=0.4 Averages 13.1 CNN & 5.2 FC
agent_cnn = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 17.5/6.0. Average score (11.5)
Win/lose count 13.5/5.0. Average score (10.0)
Win/lose count 6.0/1.0. Average score (8.333333333333334)
Win/lose count 20.5/4.0. Average score (10.375)
Win/lose count 7.0/1.0. Average score (9.5)
Win/lose count 3.0/0. Average score (8.416666666666666)
Win/lose count 10.0/3.0. Average score (8.214285714285714)
Win/lose count 19.0/5.0. Average score (8.9375)
Win/lose count 10.5/1.0. Average score (9.0)
Win/lose count 11.5/3.0. Average score (8.95)
Win/lose count 23.5/3.0. Average score (10.0)
Win/lose count 25.0/3.0. Average score (11.0)
Win/lose count 16.5/2.0. Average score (11.26923076923077)
Win/lose count 10.0/0. Average score (11.178571428571429)
Win/lose count 18.0/4.0. Average score (11.366666666666667)
Win/lose count 13.0/3.0. Average score (11.28125)
Win/lose count 16.0/3.0. Average score (11.382352941176471)
Win/lose count 12.0/2.0. Average score (11.305555555555555)
Win/lose count 27.0/2.0. Average score (12.026315

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

***
__Answer__

First,we notice that Convolutional networks gives a better Average Score (win/lose) than a classical neural network.
<br>In both videos the "rat" seems to be stuck in some area. Worse than this, he seems to like staying in the same cell. It doesn't go through the whole map. 
<br> It can be explained by the fact that epsilon is fixed or that maybe it prefers staying on his cell (where there's no penalty) than attempting to reach another cell which could contain a malus.

Concerning temperature : Increasing temperatures gives us better results on both CNN and FC (It gives a Higher probability in Bonus and in Malus, it's the p in Binomial (n,p). Hence, there are more rewards to get!


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
max_epsilon = 1
min_epsilon = 0.1 
decay_rate = 0.01



def train_explore(agent,env,epoch,prefix=''):
        # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state,train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        epsilon = min_epsilon + (max_epsilon-min_epsilon)*np.exp(decay_rate*e)
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats 
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
       
    
    pass





class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))     #Tableau avec rewards : poison -1 /cheese +0.5
        self.position = np.zeros((grid_size,grid_size))  #Position of the rat

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3)) #(502,10*16,10*16,3)


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0: #Down
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: #Up
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: #Right
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: #Left 
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1

        reward = reward + self.board[self.x, self.y]
        self.t = self.t + 1
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0] # The rat starts anywhere on the board
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2) #binomial with p=temperature
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
        
        pass
    



In [23]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.4)
agent = DQN_CNN(size, lr=.005, epsilon = 0.1, memory_size=2000, batch_size = 64,n_state=3) 
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/030 | Loss 1.7990 | Win/lose count 21.5/41.100000000000065 (-19.600000000000065)
Epoch 001/030 | Loss 1.5066 | Win/lose count 17.0/28.40000000000002 (-11.40000000000002)
Epoch 002/030 | Loss 1.2995 | Win/lose count 30.0/28.1 (1.8999999999999986)
Epoch 003/030 | Loss 1.5516 | Win/lose count 31.0/23.10000000000002 (7.899999999999981)
Epoch 004/030 | Loss 1.4455 | Win/lose count 26.5/22.5 (4.0)
Epoch 005/030 | Loss 1.6091 | Win/lose count 29.0/21.80000000000001 (7.199999999999989)
Epoch 006/030 | Loss 1.5302 | Win/lose count 29.5/21.9 (7.600000000000001)
Epoch 007/030 | Loss 1.3203 | Win/lose count 23.0/26.49999999999999 (-3.4999999999999893)
Epoch 008/030 | Loss 1.5130 | Win/lose count 31.5/25.40000000000001 (6.099999999999991)
Epoch 009/030 | Loss 1.3968 | Win/lose count 28.0/22.000000000000007 (5.999999999999993)
Epoch 010/030 | Loss 1.4836 | Win/lose count 32.0/20.500000000000004 (11.499999999999996)
Epoch 011/030 | Loss 1.3339 | Win/lose count 25.0/22.400000000000013 (2.599

In [24]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 28.5/9.0. Average score (19.5)
Win/lose count 30.0/4.0. Average score (22.75)
Win/lose count 28.0/6.0. Average score (22.5)
Win/lose count 31.5/7.0. Average score (23.0)
Win/lose count 31.5/6.0. Average score (23.5)
Win/lose count 27.0/8.0. Average score (22.75)
Win/lose count 23.5/6.0. Average score (22.0)
Win/lose count 25.0/4.0. Average score (21.875)
Win/lose count 36.0/3.0. Average score (23.11111111111111)
Win/lose count 28.5/2.0. Average score (23.45)
Win/lose count 26.5/3.0. Average score (23.454545454545453)
Win/lose count 26.5/3.0. Average score (23.458333333333332)
Win/lose count 31.0/2.0. Average score (23.884615384615383)
Win/lose count 29.0/2.0. Average score (24.107142857142858)
Win/lose count 18.0/0. Average score (23.7)
Win/lose count 36.5/4.0. Average score (24.25)
Win/lose count 28.0/4.0. Average score (24.235294117647058)
Win/lose count 21.0/4.0. Average score (23.833333333333332)
Win/lose count 32.5/4.0. Average score (24.07894736842105)
Win/lose cou

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***